In [1]:
!pip list

Package                            Version  
---------------------------------- ---------
absl-py                            0.5.0    
alabaster                          0.7.10   
anaconda-client                    1.6.14   
anaconda-project                   0.8.2    
asn1crypto                         0.24.0   
astor                              0.7.1    
astroid                            1.6.3    
astropy                            3.0.2    
attrs                              18.1.0   
Automat                            0.3.0    
autovizwidget                      0.12.5   
awscli                             1.16.75  
Babel                              2.5.3    
backcall                           0.1.0    
backports.shutil-get-terminal-size 1.0.0    
beautifulsoup4                     4.6.0    
bitarray                           0.8.1    
bkcharts                           0.2      
blaze                              0.11.3   
bleach                             2.1.3    
bokeh     

In [2]:
!pip install hyperas

    100% |████████████████████████████████| 122kB 36.5MB/s ta 0:00:01
    100% |████████████████████████████████| 829kB 27.9MB/s ta 0:00:01
    100% |████████████████████████████████| 409kB 34.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for hyperas ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/06/38/3f/27826f57fae60ef788ceb47e2c649590ab8af31f42075325d2
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built hyperas future
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip list

Package                            Version  
---------------------------------- ---------
absl-py                            0.5.0    
alabaster                          0.7.10   
anaconda-client                    1.6.14   
anaconda-project                   0.8.2    
asn1crypto                         0.24.0   
astor                              0.7.1    
astroid                            1.6.3    
astropy                            3.0.2    
attrs                              18.1.0   
Automat                            0.3.0    
autovizwidget                      0.12.5   
awscli                             1.16.75  
Babel                              2.5.3    
backcall                           0.1.0    
backports.shutil-get-terminal-size 1.0.0    
beautifulsoup4                     4.6.0    
bitarray                           0.8.1    
bkcharts                           0.2      
blaze                              0.11.3   
bleach                             2.1.3    
bokeh     

In [4]:
pwd

'/home/ec2-user/SageMaker'

In [5]:
import os
os.listdir('/home/ec2-user/SageMaker')

['__pycache__',
 '.ipynb_checkpoints',
 'Hyperas_mjy_v1_3.ipynb',
 '.sparkmagic',
 'lost+found',
 '.bashrc',
 '.profile']

In [6]:
# Keras Imports
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence 
from keras.wrappers.scikit_learn import KerasClassifier
# Numpy
import numpy
numpy.random.seed(1331)
# Pandas
import pandas as pd
# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# Visualizations
import matplotlib.pyplot as plt
%matplotlib inline
# Garbage Collector
import gc
import sys
# Boto is the Amazon Web Services (AWS) SDK for Python, which allows Python developers to write software that makes 
# use of Amazon services like S3 and EC2. Boto provides an easy to use, object-oriented API as well as low-level direct 
# service access.
import boto3

Using TensorFlow backend.


In [7]:
# Hyperas Imports
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

In [16]:
import pickle

In [8]:
# Get Data
def data():
    bucket='quora-insincere-mjy' # Or whatever you called your bucket
    data_key = 'train.csv' # Where the file is within your bucket
    data_location = 's3://{}/{}'.format(bucket, data_key)
    q_df = pd.read_csv(data_location)
    x_train, x_test, y_train, y_test = train_test_split(q_df['question_text'], q_df['target'], test_size=0.50, random_state=1331)
    # create a tokenizer 
    token = keras.preprocessing.text.Tokenizer()
    token.fit_on_texts(q_df['question_text'])
    word_index = token.word_index
    # convert text to sequence of tokens and pad them to ensure equal length vectors 
    max_review_length = 134
    x_train = sequence.pad_sequences(token.texts_to_sequences(x_train), maxlen=max_review_length)
    x_test = sequence.pad_sequences(token.texts_to_sequences(x_test), maxlen=max_review_length)
    return x_train, y_train, x_test, y_test

In [9]:
# Function to create model, required for KerasClassifier
def create_model(x_train, y_train, x_test, y_test):
    # create model
    model = Sequential()
    model.add(Embedding(input_dim={{choice([1000, 5088, 10000, 50882, 100000, 508823])}}, 
                        output_dim={{choice([10, 15, 20, 25, 30, 35, 40, 45, 50])}}, 
                        input_length=134))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Conv1D(filters={{choice([35, 40, 45, 50, 55, 60, 65, 70])}}, 
                     kernel_size={{choice([1, 2, 3, 4, 5])}}, 
                     padding='same', 
                     activation={{choice(['relu','tanh'])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(MaxPooling1D(pool_size={{choice([1, 2, 3, 4, 5])}}))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add((CuDNNLSTM({{choice([20, 25, 30, 35, 40])}})))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense(1, activation='sigmoid'))
    
    adam=keras.optimizers.Adam(lr={{uniform(0,1)}})
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size={{choice([100, 1000, 10000])}},
              nb_epoch={{choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])}},
              verbose=2,
              validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test, verbose=0)
    print('Test accuracy:', acc)
    
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [10]:
try:
    bucket_r='quora-insincere-mjy' # Or whatever you called your bucket
    data_key_r = 'result_custom.pkl' # Where the file is within your bucket
    data_location_r = 's3://{}/{}'.format(bucket_r, data_key_r)
    trials = pickle.load(open(data_location_r, "rb"))
    print("Found saved Trials! Loading...")
    max_evals = len(trials.trials) + nb_evals
    print("Rerunning from {} trials to add more.".format(len(trials.trials)))
except:
    trials = Trials()
    print("Starting from scratch: new trials.")

Starting from scratch: new trials.


In [12]:
# Run Hyperas
x_train, y_train, x_test, y_test = data()

best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=20,
                                      trials=trials,
                                      eval_space=True,
                                      notebook_name='Hyperas_mjy_v1_3')

>>> Imports:
#coding=utf-8

try:
    import os
except:
    pass

try:
    import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense
except:
    pass

try:
    from keras.layers import LSTM
except:
    pass

try:
    from keras.layers import CuDNNLSTM
except:
    pass

try:
    from keras.layers import Conv1D
except:
    pass

try:
    from keras.layers import MaxPooling1D
except:
    pass

try:
    from keras.layers import Dropout
except:
    pass

try:
    from keras.layers.embeddings import Embedding
except:
    pass

try:
    from keras.preprocessing import sequence
except:
    pass

try:
    from keras.wrappers.scikit_learn import KerasClassifier
except:
    pass

try:
    import numpy
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.metrics import f1_score
except:
    pass

try:
    from sk

/home/ec2-user/SageMaker/temp_model.py:166: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 653061 samples, validate on 653061 samples
Epoch 1/2
 - 62s - loss: 0.3185 - acc: 0.9119 - val_loss: 0.2341 - val_acc: 0.9383
Epoch 2/2
 - 26s - loss: 0.2330 - acc: 0.9380 - val_loss: 0.2318 - val_acc: 0.9383
Test accuracy: 0.9382523225243271
Train on 653061 samples, validate on 653061 samples
Epoch 1/7
 - 40s - loss: 0.9975 - acc: 0.9370 - val_loss: 0.9953 - val_acc: 0.9383
Epoch 2/7
 - 38s - loss: 0.9992 - acc: 0.9380 - val_loss: 0.9953 - val_acc: 0.9383
Epoch 3/7
 - 38s - loss: 0.9992 - acc: 0.9380 - val_loss: 0.9953 - val_acc: 0.9383
Epoch 4/7
 - 38s - loss: 0.9992 - acc: 0.9380 - val_loss: 0.9953 - val_acc: 0.9383
Epoch 5/7
 - 38s - loss: 0.9992 - acc: 0.9380 - val_loss: 0.9953 - val_acc: 0.9383
Epoch 6/7
 - 38s - loss: 0.9992 - acc: 0.9380 - val_loss: 0.9953 - val_acc: 0.9383
Epoch 7/7
 - 38s - loss: 0.9992 - acc: 0.9380 - val_loss: 0.9953 - val_acc: 0.9383
Test accuracy: 0.9382523225243271
Train on 653061 samples, validate on 653061 samples
Epoch 1/3
 - 41s - loss: 0.23

In [13]:
# Test Best Model
print("Evalutation of best performing model:")
print(best_model.evaluate(x_test, y_test))

Evalutation of best performing model:
653061/653061 [==============================] - 91s 140us/step
[0.2368299171066441, 0.9382676350295908]


In [14]:
# Show Best Model
print(best_run)

{'CuDNNLSTM': 25, 'Dropout': 0.7871478922143195, 'Dropout_1': 0.21887493324099216, 'Dropout_2': 0.25634389232531474, 'Dropout_3': 0.12064860770171626, 'activation': 'tanh', 'batch_size': 100, 'filters': 45, 'input_dim': 100000, 'kernel_size': 4, 'kernel_size_1': 3, 'lr': 0.5350807190884803, 'nb_epoch': 14, 'output_dim': 35}


In [32]:
!pip install s3io

  Running setup.py bdist_wheel for s3io ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/93/8b/6d/8c5cd21b57cae3fb88345f5621e22ebe432bee9de3202dfc26
Successfully built s3io
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [34]:
!pip install joblib

    100% |████████████████████████████████| 276kB 29.9MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [83]:
"""Example of usage of Joblib with Amazon S3."""

import s3io
import joblib
import numpy as np
    
big_obj =  trials.trials

# Customize the following values with yours
bucket = 'quora-insincere-mjy'
key = 'result_custom.pkl'

# Dump in an S3 file is easy with Joblib
with s3io.open('s3://{0}/{1}'.format(bucket, key), mode='w') as s3_file:
    joblib.dump(big_obj, s3_file)

with s3io.open('s3://{0}/{1}'.format(bucket, key), mode='r') as s3_file:
    obj_reloaded = joblib.load(s3_file)

print("Correctly reloaded? {0}".format(all(np.allclose(x, y)
                                           for x, y in zip(big_obj,
                                                           obj_reloaded))))

TypeError: can't pickle _thread.RLock objects